In [ ]:
! pip install datasets transformers rouge-score nltk
!apt install git-lfs


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 85.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


#Fine-tuning a model on a summarization task


In [ ]:
import transformers

print(transformers.__version__)

4.28.1


In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("summarization_test", framework="pytorch")

In [ ]:
model_checkpoint = "t5-small"


In [ ]:
from datasets import load_dataset, load_metric
from datasets.dataset_dict import DatasetDict

#raw_datasets = load_dataset("xsum")

raw_datasets = DatasetDict({"train": load_dataset("xsum", split='train[1%:2%]'),
                            "validation": load_dataset("xsum", split='validation[1%:2%]'),
                            "test": load_dataset("xsum", split='test[1%:2%]')})
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 2041
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 114
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 114
    })
})

In [ ]:
from datasets import load_metric

metric = load_metric("rouge")

<ipython-input-6-1a0e7328e82b>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [ ]:
raw_datasets


DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 2041
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 114
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 114
    })
})

In [ ]:
raw_datasets["train"][0]


{'document': 'The 53-year-old, who has also managed Millwall and Sunderland, was sacked as Wolves boss in February.\nHe will be assisted by his former number two at Wolves, Terry Connor, who succeeded him as manager at Molineux.\nMcCarthy replaces Paul Jewell,  with the club bottom of the Championship table.\nMick McCarthy has managed all his three clubs (Millwall, Sunderland and Wolves) to the First Division/Championship play-offs and led the latter two up to the Premier League as champions\nJewell\'s assistant Chris Hutchings, who took charge of Saturday\'s 3-0 loss to Sheffield Wednesday - Town\'s fourth straight loss and 12th game without a win - has left the cub by mutual consent.\n"I\'m obviously delighted to have been given the job as Ipswich Town manager and I\'m looking forward to the challenge ahead," McCarthy told the club website.\n"It\'s a fantastic football club, with a proud tradition and history and a terrific fanbase. While the long term ambition is to take the club ba

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])


In [ ]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

In [ ]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

#Preprocessing the data


In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer("hello folks")


{'input_ids': [21820, 5265, 1], 'attention_mask': [1, 1, 1]}

In [ ]:
tokenizer(["hello folks", "hello friends"])


{'input_ids': [[21820, 5265, 1], [21820, 803, 1]], 'attention_mask': [[1, 1, 1], [1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["hello folks", "hello friends"]))

{'input_ids': [[21820, 5265, 1], [21820, 803, 1]], 'attention_mask': [[1, 1, 1], [1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(raw_datasets['train'][:2])


{'input_ids': [[21603, 10, 37, 12210, 18, 1201, 18, 1490, 6, 113, 65, 92, 3030, 4185, 10279, 11, 3068, 221, 7721, 6, 47, 3, 7, 13365, 38, 3488, 8391, 7930, 16, 2083, 5, 216, 56, 36, 11752, 57, 112, 1798, 381, 192, 44, 3488, 8391, 6, 18163, 3, 22329, 6, 113, 22399, 376, 38, 2743, 44, 11019, 77, 2623, 5, 30757, 3601, 7, 1838, 18869, 40, 6, 28, 8, 1886, 2007, 13, 8, 7666, 953, 5, 25665, 30757, 65, 3030, 66, 112, 386, 8122, 41, 329, 1092, 10279, 6, 3068, 221, 7721, 11, 3488, 8391, 61, 12, 8, 1485, 6022, 87, 254, 1483, 12364, 2009, 577, 18, 1647, 7, 11, 2237, 8, 7722, 192, 95, 12, 8, 6552, 3815, 38, 6336, 7, 18869, 40, 31, 7, 6165, 4409, 3455, 17, 8509, 7, 6, 113, 808, 1567, 13, 1856, 31, 7, 3, 22773, 1453, 12, 23826, 2875, 3, 18, 4463, 31, 7, 4509, 2541, 1453, 11, 586, 189, 467, 406, 3, 9, 1369, 3, 18, 65, 646, 8, 123, 115, 57, 8543, 6641, 5, 96, 196, 31, 51, 6865, 10693, 12, 43, 118, 787, 8, 613, 38, 27, 102, 7, 210, 362, 4463, 2743, 11, 27, 31, 51, 479, 1039, 12, 8, 1921, 2177, 976, 3075

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)


#Fine-tuning the model


In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-test",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    # push_to_hub=True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.892678,19.326100,3.772700,15.325900,15.366800,18.149100


TrainOutput(global_step=128, training_loss=3.1116445064544678, metrics={'train_runtime': 106.5121, 'train_samples_per_second': 19.162, 'train_steps_per_second': 1.202, 'total_flos': 545012975665152.0, 'train_loss': 3.1116445064544678, 'epoch': 1.0})

In [ ]:
# trainer.push_to_hub()
